In [1]:
import numpy as np
from MyGenerator import DataGenerator
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
import pickle
import os

Using TensorFlow backend.


In [2]:
with open(os.path.join("regression_data", 'labels.pickle'), 'rb') as handle:
    labels = pickle.load(handle)

In [3]:
list_of_labels = [*labels]
np.random.shuffle(list_of_labels)

In [4]:
len(labels)

50000

In [8]:
partition = {'train': list_of_labels[:40000], 
             'validation': list_of_labels[40000:45000],
             'test': list_of_labels[45000:50000]}
y_label = {'train': [labels[k] for k in partition['train']], 
           'validation': [labels[k] for k in partition['validation']],
           'test': [labels[k] for k in partition['test']]}

In [9]:
len(y_label['validation'])

5000

In [10]:
len(partition['validation'])

5000

In [11]:
params = {'dim': (2500,),
          'batch_size': 32,
          'n_channels': 1,
          'shuffle': True}
test_params = {'dim': (2500,),
               'batch_size': len(partition['test']),
               'n_channels': 1,
               'shuffle': False}

In [12]:
training_generator = DataGenerator(partition['train'], y_label['train'],  **params)
validation_generator = DataGenerator(partition['validation'], y_label['validation'],  **params)
testing_generator = DataGenerator(partition['test'], y_label['test'],  **test_params)

In [13]:
inputs = keras.Input(shape = (2500,1))
conv_1_1 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(inputs)
conv_1_2 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_1)
conv_1_3 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_2)
pool_1 = keras.layers.MaxPool1D(pool_size = 4)(conv_1_3)

conv_2_1 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_1)
conv_2_2 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_2_1)
#conv_2_3 = keras.layers.Conv1D(64, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_2_2)
pool_2 = keras.layers.MaxPool1D(pool_size = 4)(conv_2_2)

conv_3_1 = keras.layers.Conv1D(128, kernel_size = 7, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_2)
#conv_3_2 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_1)
#conv_3_3 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_2)
pool_3 = keras.layers.MaxPool1D(pool_size = 4)(conv_3_1)



flat_1 = keras.layers.Flatten()(pool_3)



dense_1 = keras.layers.Dense(1024, activation = 'relu', 
                             kernel_initializer = keras.initializers.he_normal())(flat_1)
outputs = keras.layers.Dense(3, kernel_initializer = keras.initializers.he_normal())(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2500, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2498, 32)          128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2496, 32)          3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2494, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 623, 32)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 619, 64)           10304     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 615, 64)           20544     
__________

In [14]:
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.roof_reg.hdf5', 
                               verbose=2, save_best_only=True)
model.compile(optimizer=adam,
              loss='mean_squared_error', metrics = ['mae'])
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              #steps_per_epoch=training_generator.__len__(),
                              #workers=16,
                              callbacks=[checkpointer],
                              epochs = 100,
                              use_multiprocessing=True)

Epoch 1/100
1250/1250 [==============================] - 24s 19ms/step - loss: 6.0151 - mean_absolute_error: 0.9122 - val_loss: 0.9053 - val_mean_absolute_error: 0.5015

Epoch 00001: val_loss improved from inf to 0.90533, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 2/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.7165 - mean_absolute_error: 0.4329 - val_loss: 0.5825 - val_mean_absolute_error: 0.4014

Epoch 00002: val_loss improved from 0.90533 to 0.58250, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 3/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.5569 - mean_absolute_error: 0.3683 - val_loss: 0.5336 - val_mean_absolute_error: 0.3488

Epoch 00003: val_loss improved from 0.58250 to 0.53361, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 4/100
1250/1250 [==============================] - 18s 14ms/step - loss: 0.4750 - mean_absolute_error: 0.3331 - val_loss: 0.4029 - val_mean_absolute_err

1250/1250 [==============================] - 17s 14ms/step - loss: 0.1129 - mean_absolute_error: 0.1500 - val_loss: 0.1562 - val_mean_absolute_error: 0.1651

Epoch 00065: val_loss did not improve from 0.15511
Epoch 66/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.1119 - mean_absolute_error: 0.1496 - val_loss: 0.1591 - val_mean_absolute_error: 0.1666

Epoch 00066: val_loss did not improve from 0.15511
Epoch 67/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.1118 - mean_absolute_error: 0.1493 - val_loss: 0.1572 - val_mean_absolute_error: 0.1643

Epoch 00067: val_loss did not improve from 0.15511
Epoch 68/100
1250/1250 [==============================] - 17s 14ms/step - loss: 0.1091 - mean_absolute_error: 0.1486 - val_loss: 0.1542 - val_mean_absolute_error: 0.1642

Epoch 00068: val_loss improved from 0.15511 to 0.15417, saving model to saved_models/weights.best.roof_reg.hdf5
Epoch 69/100
1250/1250 [==============================] - 17s 14

In [15]:
model.load_weights('saved_models/weights.best.roof_reg.hdf5')
model.evaluate_generator(testing_generator)

[0.14824512600898743, 0.1600790023803711]

In [16]:
partition['test'][4]

'roof_15145'

In [17]:
y_label['test'][4]

[0.9182646253903681, -9.259415400404274, 3.369987760093432]

In [18]:
k = 45
X_test = np.zeros((1,2500,1))
X_test[0,] = np.load("regression_data/" + partition['test'][k] + '.npy')[:,np.newaxis]

In [19]:
print(model.predict(X_test), y_label['test'][k] )

[[7.3826365 9.1093235 3.4070392]] [7.265479012812818, 9.089658725624282, 3.5855657503976275]


In [20]:
model.save('saved_models/roof_model.h5')